  # PostProcessing
  This notebook is initially designed for PostProcessing ASETS-II numerical and experimental data.
  It can also be used for other configurations's numerical results. But if you want to compare with other experimental data with a different format than ASETS-II. You should find a way to read them to RTD and RTDt in this notebook.

In [2]:
using ComputationalHeatTransfer
using Plots
# using Interact
using Statistics
using EasyFit
import XLSX

In [68]:
dir_paths = [
    "/Users/yuxuanli/Documents/Hoffman/ohp162/numedata";
    "/Users/yuxuanli/Documents/Hoffman/ohp171/numedata";
    "/Users/yuxuanli/Documents/Hoffman/ohp172/numedata";
    "/Users/yuxuanli/Documents/Hoffman/ohp173/numedata";
    "/Users/yuxuanli/Documents/Hoffman/ohp174/numedata";
    "/Users/yuxuanli/Documents/Hoffman/ohp175/numedata";
    "/Users/yuxuanli/Documents/Hoffman/ohp176/numedata";
]
file_path = "/ASETS-L-2-40.jld2"

"/ASETS-L-2-40.jld2"

In [69]:
x2Dsensors = [-2.9,-1.4,-0.8,0.0,0.0,0.8,1.4,2.9] .* inches
y2Dsensors = [0.0,   0.0, 0.0,0.0,0.4,0.0,0.0,0.0] .* inches
plate_sensors = (x2Dsensors,y2Dsensors);

In [70]:
cond_all = []
dT_all = []
plt = plot()
for dir_path in dir_paths
    read_path = dir_path*file_path
    SimuResult = load(read_path)["SimulationResult"];
    t_hist,g_hist = getTcurve(plate_sensors,SimuResult);
    
    ihot = 4 # hot sensor  for calculating thermal conductance
    icold = 8# cold sensor  for calculating thermal conductance;
    power = SimuResult.integrator_tube.p.wall.power
    
    # push!(cond_all,power./(g_hist[:,ihot] - g_hist[:,icold]);)
    push!(dT_all,(g_hist[:,ihot] - g_hist[:,icold]))
    
#     plot!(OHPCond(),(ihot,icold),(t_hist,g_hist),(RTDt,RTD),SimuResult,ylim=(0,15))
end

In [73]:
# cond_max = maximum(hcat(cond_all...),dims=2);
# cond_min = minimum(hcat(cond_all...),dims=2);

dT_max = maximum(hcat(dT_all...),dims=2);
dT_min = minimum(hcat(dT_all...),dims=2);

In [74]:
dir_path = dir_paths[1]
read_path = dir_path*file_path
    SimuResult = load(read_path)["SimulationResult"];
    t_hist,g_hist = getTcurve(plate_sensors,SimuResult);

    cd("/Users/yuxuanli/Documents/Hoffman/ohp95/numedata")
    expfile = expfileDict["O002_H001_P040"]
    exppath = "../expdata/"
    xf = XLSX.readxlsx(exppath*expfile);
    Onum, Hnum, power_exp = getconfig(expfile)
    RTDt,RTD = getRTD(xf,Onum);

In [75]:
ihot = 4 # hot sensor  for calculating thermal conductance
    icold = 8# cold sensor  for calculating thermal conductance;
    power = SimuResult.integrator_tube.p.wall.power

cond = power./(g_hist[:,ihot] - g_hist[:,icold]);
dT = g_hist[:,ihot] - g_hist[:,icold];
dt_exp = RTD[:,ihot]-RTD[:,icold];

In [76]:
if length(SimuResult.boil_hist) != 0
boil_data,boil_num_x,boil_num_t,t_boil,x2D_boil,y2D_boil,boil_dt = get_boil_matrix(SimuResult::SimulationResult);
end

([88.0 2.7499999999997624; 36.0 4.7599999999995415; … ; 10.0 234.55000000329213; 241.0 258.9900000041034], [5.0; 8.0; … ; 0.0; 0.0;;], [0.0; 0.0; … ; 0.0; 1.0;;], 0.1:0.1:259.0, [-0.041266614379100036, 0.007159650837879283, 0.016018438816322977, 0.018697185099937865, 0.028623112601812897, 0.03155675557471656, 0.04220363461263709, 0.057544726761539045, 0.01927919132375805, 0.002781309321315029  …  0.016643962656571136, 0.021413585668903248, 0.06615375314770464, 0.05453235914779503, 0.009564775717435248, -0.027722698256917165, -0.0347807157968054, -0.035780824010705904, -0.068915, -0.068915], [-0.023125, -0.023125, -0.023125, -0.023125, -0.023125000000000003, -0.023125, -0.023125, -0.023125, -0.021723484848484843, -0.02172348484848484  …  0.021723484848485124, 0.021723484848485124, 0.021723484848485124, 0.023125000000000277, 0.023125000000000277, 0.02312500000000029, 0.02312500000000029, 0.02312500000000029, 0.020912048077899627, -0.008446356850664829], 0.1)

In [77]:
m10 = movavg(vcat(boil_num_t...)./boil_dt,10);

In [81]:
gr()
# plot(OHPCond(),(ihot,icold),(t_hist,g_hist),(RTDt,RTD),SimuResult,ylim=(0,5))
plot(t_hist,dT,xlim=(0,300.0),ylim=(-20,100),linewidth=2,label="T₄-T₈ model",ylabel="T₄-T₈ [K]",legend=:topleft, xlabel="time [s]")
scatter!(RTDt,dt_exp,label="T₄-T₈ experiment")
plot!(dT_max,fillrange = dT_min,fillalpha = 0.35,fillcolor=:blue,linewidth=0,alpha=0,label=false,title="")
plot!(twinx(),t_boil,m10.x,
color=:orange, xlim=(0,300.0),ylim=(0,100),legend=:top,linewidth=2, ylabel="f [Hz]", label="boiling frequency")
# savefig("cond_curve.pdf")
savefig("cond_curve dryout.pdf")

"/Users/yuxuanli/Documents/Hoffman/ohp95/numedata/cond_curve dryout.pdf"

## average conductance

In [83]:
read_path_array = [
"../numedata/ASETS-L-1-10.jld2",
"../numedata/ASETS-L-1-20.jld2",
"../numedata/ASETS-L-1-30.jld2",
"../numedata/ASETS-L-1-40.jld2",
"../numedata/ASETS-S-1-10.jld2",
"../numedata/ASETS-S-1-20.jld2",
"../numedata/ASETS-S-1-30.jld2",
"../numedata/ASETS-S-1-40.jld2",
"../numedata/ASETS-L-2-10.jld2",
"../numedata/ASETS-L-2-20.jld2",
"../numedata/ASETS-L-2-30.jld2",
"../numedata/ASETS-L-2-40.jld2",
"../numedata/ASETS-S-2-10.jld2",
"../numedata/ASETS-S-2-20.jld2",
"../numedata/ASETS-S-2-30.jld2",
"../numedata/ASETS-S-2-40.jld2"
]
exp_path_array = [
    "O001_H001_P010",
    "O001_H001_P020",
    "O001_H001_P030",
    "O001_H001_P040",
    "O001_H002_P010",
    "O001_H002_P020",
    "O001_H002_P030",
    "O001_H002_P040",
    "O002_H001_P010",
    "O002_H001_P020",
    "O002_H001_P030",
    "O002_H001_P040",
    "O002_H002_P010",
    "O002_H002_P020",
    "O002_H002_P030",
    "O002_H002_P040"
]

exppath = "/Users/yuxuanli/Documents/Hoffman/ohp95/expdata/"

function getallcond(readpath,read_path_array,exppath,exp_path_array,plate_sensors)
    cd(readpath)
    
    conductance_array = zeros(size(read_path_array))
    expcond_array = zeros(size(read_path_array))
    avgδ_array = zeros(size(read_path_array))
    nume_postdryout = 0.0
    exp_postdryout = 0.0

    for i in eachindex(read_path_array)
        SimuResult = load(read_path_array[i])["SimulationResult"];
        
        t_hist,g_hist = getTcurve(plate_sensors,SimuResult);
        power = SimuResult.integrator_tube.p.wall.power

        println(i,power)

        ihot = 4 # hot sensor  for calculating thermal conductance
        icold = 8 # c
        cond = power ./ (g_hist[:,ihot] .- g_hist[:,icold])

        conductance_array[i] = mean(cond[100:200])
#         avgδ_array[i] = get_avgδ(SimuResult)


        expfile = expfileDict[exp_path_array[i]]
#         exppath = "/Users/yuxuanli/Documents/Hoffman/ohp95/expdata/"
        xf = XLSX.readxlsx(exppath*expfile);

        Onum, Hnum, power_exp = getconfig(expfile)
        RTDt,RTD = getRTD(xf,Onum);

        cond_exp = power ./ (RTD[:,ihot] .- RTD[:,icold])

        expcond_array[i] = mean(cond_exp[100:200])

        if exp_path_array[i] == "O002_H001_P040"
            exp_postdryout = mean(cond_exp[250:300])
            nume_postdryout = mean(cond[250:300])
        end
    end

    return conductance_array,expcond_array,exp_postdryout,nume_postdryout
end

getallcond (generic function with 1 method)

In [84]:
conductance_array_all = []
for readpath in dir_paths
    conductance_array,expcond_array,exp_postdryout,nume_postdryout = getallcond(readpath,read_path_array,exppath,exp_path_array,plate_sensors);
    push!(conductance_array_all,conductance_array)
end

110.0
220.0
330.0
440.0
510.0
620.0
730.0
840.0
910.0
1020.0
1130.0
1240.0
1310.0
1420.0
1530.0
1640.0
110.0
220.0
330.0
440.0
510.0
620.0
730.0
840.0
910.0
1020.0
1130.0
1240.0
1310.0
1420.0
1530.0
1640.0
110.0
220.0
330.0
440.0
510.0
620.0
730.0
840.0
910.0
1020.0
1130.0
1240.0
1310.0
1420.0
1530.0
1640.0
110.0
220.0
330.0
440.0
510.0
620.0
730.0
840.0
910.0
1020.0
1130.0
1240.0
1310.0
1420.0
1530.0
1640.0
110.0
220.0
330.0
440.0
510.0
620.0
730.0
840.0
910.0
1020.0
1130.0
1240.0
1310.0
1420.0
1530.0
1640.0
110.0
220.0
330.0
440.0
510.0
620.0
730.0
840.0
910.0
1020.0
1130.0
1240.0
1310.0
1420.0
1530.0
1640.0
110.0
220.0
330.0
440.0
510.0
620.0
730.0
840.0
910.0
1020.0
1130.0
1240.0
1310.0
1420.0
1530.0
1640.0


In [40]:
readpath = "/Users/yuxuanli/Documents/Hoffman/ohp162/numedata"
conductance_array,expcond_array,exp_postdryout,nume_postdryout = getallcond(readpath,read_path_array,exppath,exp_path_array,plate_sensors);

110.0
220.0
330.0
440.0
510.0
620.0
730.0
840.0
910.0
1020.0
1130.0
1240.0
1310.0
1420.0
1530.0
1640.0


In [85]:
conductance_array = vec(mean(hcat(conductance_array_all...),dims=2))
cond_err_max = vec(maximum(hcat(conductance_array_all...),dims=2) .- conductance_array)
cond_err_min = vec(-(minimum(hcat(conductance_array_all...),dims=2) .- conductance_array));
# σavg = std(hcat(conductance_array_all...),dims=2);

In [86]:
conductance_array[plotrange]

4-element Vector{Float64}:
 3.7642465150405777
 6.303638108765004
 6.233366401506048
 5.792706013890706

In [87]:
cd("/Users/yuxuanli/Documents/Hoffman/ohp95/numedata/")

In [96]:
powers = [10,20,30,40]
plotrange = 1:4
scatter(powers,conductance_array[plotrange],markersize=5,color=:blue,markeralpha=0.5,marker=:square,yerror=(cond_err_min[plotrange],cond_err_max[plotrange]))
scatter!(powers,expcond_array[plotrange],markersize=5,markeralpha=0.5,color=:red,
ylim=(0,8),legend=false,xlabel="heater power [W]",ylabel="G [W/K]")
savefig("cond all 1L.pdf")

"/Users/yuxuanli/Documents/Hoffman/ohp95/numedata/cond all 1L.pdf"

In [97]:
powers = [10,20,30,40]
plotrange = 5:8
scatter(powers,conductance_array[plotrange],markersize=5,markeralpha=0.5,color=:blue,marker=:square,yerror=(cond_err_min[plotrange],cond_err_max[plotrange]))
scatter!(powers,expcond_array[plotrange],markersize=5,markeralpha=0.5,color=:red,
ylim=(0,8),legend=false,xlabel="heater power [W]",ylabel="G [W/K]")
savefig("cond all 1S.pdf")

"/Users/yuxuanli/Documents/Hoffman/ohp95/numedata/cond all 1S.pdf"

In [98]:
powers = [10,20,30,40]
plotrange = 9:12
scatter(powers,conductance_array[plotrange],markersize=5,markeralpha=0.5,color=:blue,marker=:square,yerror=(cond_err_min[plotrange],cond_err_max[plotrange]),label="")
scatter!(powers,expcond_array[plotrange],markersize=5,markeralpha=0.5,color=:red,label="",
ylim=(0,8),legend=true,xlabel="heater power [W]",ylabel="G [W/K]")
plot!([40,40],[expcond_array[12],exp_postdryout],markersize=5,markeralpha=0.5,linewidth=2,marker=:xcross,color=:red,label="experimental dryout")
# scatter!([40],[nume_postdryout],markersize=5,markeralpha=0.5,color=:blue,marker=:square,legend=true,label="")
savefig("cond all 2L.pdf")

"/Users/yuxuanli/Documents/Hoffman/ohp95/numedata/cond all 2L.pdf"

In [99]:
powers = [10,20,30,40]
plotrange = 13:16
scatter(powers,conductance_array[plotrange],markersize=5,markeralpha=0.5,color=:blue,marker=:square,yerror=(cond_err_min[plotrange],cond_err_max[plotrange]))
scatter!(powers,expcond_array[plotrange],markersize=5,markeralpha=0.5,color=:red,
ylim=(0,8),legend=false,xlabel="heater power [W]",ylabel="G [W/K]")
savefig("cond all 2S.pdf")

"/Users/yuxuanli/Documents/Hoffman/ohp95/numedata/cond all 2S.pdf"

In [95]:
maximum(abs.(1 .- [conductance_array[1]./expcond_array[1]]))

0.23780132361691642

#### 